# AUA, DS 229 – MLOps
### Week 2 – Responsible Machine Learning with Error Analysis

- [Responsible Machine Learning with Error Analysis](https://techcommunity.microsoft.com/t5/ai-machine-learning-blog/responsible-machine-learning-with-error-analysis/ba-p/2141774)
- [Responsible AI dashboard: A one-stop shop for operationalizing Responsible AI in practice](https://techcommunity.microsoft.com/t5/ai-machine-learning-blog/responsible-ai-dashboard-a-one-stop-shop-for-operationalizing/ba-p/3030944)
- [AI Show: Live Responsible AI Dashboard: One-stop shop for operationalizing RAI in practice](https://techcommunity.microsoft.com/t5/video-hub/ai-show-live-responsible-ai-dashboard-one-stop-shop-for/ba-p/3060153)
- [Getting started](https://github.com/microsoft/responsible-ai-toolbox/blob/main/notebooks/responsibleaidashboard/getting-started.ipynb)
- [Take a Tour: Responsible AI Toolbox](https://github.com/microsoft/responsible-ai-toolbox/blob/main/notebooks/responsibleaidashboard/tour.ipynb)

# Notebook 02 – Debug housing price predictions

This notebook demonstrates the use of the Responsible API to assess a classification model trained on Kaggle's apartments dataset (https://www.kaggle.com/alphaepsilon/housing-prices-dataset). The model predicts if the house sells for more than median price or not. It walks through the API calls necessary to create a widget with model analysis insights, then guides a visual analysis of the model.

In [ ]:
# !pip install --upgrade pip
# !pip install interpret-community 
# !pip install raiwidgets

## Launch Responsible AI Toolbox

The following section examines the code necessary to create datasets and a model. It then generates insights using the `responsibleai` API that can be visually analyzed.

### Train a Model
*The following section can be skipped. It loads a dataset and trains a model for illustrative purposes.*

In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
import zipfile

First, load the apartment dataset and specify the different types of features. Then, clean it and put it into a DataFrame with named columns. After loading and cleaning the data, split the datapoints into training and test sets. Assemble separate datasets for the full sample and the test data.

In [ ]:
from raiutils.dataset import fetch_dataset
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer


def split_label(dataset, target_feature):
    X = dataset.drop([target_feature], axis=1)
    y = dataset[[target_feature]]
    return X, y


def clean_data(X, y, target_feature):
    features = X.columns.values.tolist()
    classes = y[target_feature].unique().tolist()
    pipe_cfg = {
        'num_cols': X.dtypes[X.dtypes == 'int64'].index.values.tolist(),
        'cat_cols': X.dtypes[X.dtypes == 'object'].index.values.tolist(),
    }
    num_pipe = Pipeline([
        ('num_imputer', SimpleImputer(strategy='median'))
    ])
    cat_pipe = Pipeline([
        ('cat_imputer', SimpleImputer(strategy='constant', fill_value='?')),
        ('cat_encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))
    ])
    feat_pipe = ColumnTransformer([
        ('num_pipe', num_pipe, pipe_cfg['num_cols']),
        ('cat_pipe', cat_pipe, pipe_cfg['cat_cols'])
    ])
    X = feat_pipe.fit_transform(X)
    print(pipe_cfg['cat_cols'])
    return X, feat_pipe, features, classes

In [ ]:
target_feature = 'Sold_HigherThan_Median'
categorical_features = []

outdirname = 'responsibleai.12.28.21'
zipfilename = outdirname + '.zip'

fetch_dataset('https://publictestdatasets.blob.core.windows.net/data/' + zipfilename, zipfilename)

with zipfile.ZipFile(zipfilename, 'r') as unzip:
    unzip.extractall('.')

all_data = pd.read_csv('apartments-train.csv')
all_data = all_data.drop(['SalePrice','SalePriceK'], axis=1)
X, y = split_label(all_data, target_feature)


X_train_original, X_test_original, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=7, stratify=y)

X_train, feat_pipe, features, classes = clean_data(X_train_original, y_train, target_feature)
y_train = y_train[target_feature].to_numpy()

X_test = feat_pipe.transform(X_test_original)
y_test = y_test[target_feature].to_numpy()

train_data = X_train_original.copy()
train_data[target_feature] = y_train

test_data = X_test_original.copy()
test_data[target_feature] = y_test

Train a LightGBM classifier on the training data.

In [ ]:
clf = LGBMClassifier(random_state=0)
model = clf.fit(X_train, y_train)

### Create Model and Data Insights

In [ ]:
from raiwidgets import ResponsibleAIDashboard
from responsibleai import RAIInsights

To use Responsible AI Dashboard, initialize a RAIInsights object upon which different components can be loaded.

RAIInsights accepts the model, the full dataset, the test dataset, the target feature string, the task type string, and a list of strings of categorical feature names as its arguments.

In [ ]:
from sklearn.pipeline import Pipeline

dashboard_pipeline = Pipeline(steps=[('preprocess', feat_pipe), ('model', model)])
rai_insights = RAIInsights(dashboard_pipeline, train_data, test_data, target_feature, 'classification',
                           categorical_features=categorical_features, 
                           classes=['Less than median', 'More than median'])

Add the components of the toolbox that are focused on model assessment.

In [ ]:
rai_insights.explainer.add()  # Interpretability.
rai_insights.error_analysis.add()  # Error Analysis.

# Counterfactuals: accepts total number of counterfactuals to generate, the label that they should have, and a list of 
#                  strings of categorical feature names
rai_insights.counterfactual.add(total_CFs=10, desired_class='opposite')

Once all the desired components have been loaded, compute insights on the test set.

In [ ]:
rai_insights.compute()

Finally, visualize and explore the model insights. Use the resulting widget or follow the link to view this in a new tab.

In [ ]:
ResponsibleAIDashboard(rai_insights, port=7009)

#### **Tasks:**
<div class="alert alert-success">

✅ Investigate **Error analysis** tab (see Notebook 01 – Regression Error Analysis for more information).
    
✅ Investigate **Model overview** tab (see Notebook 01 – Regression Error Analysis for more information).

✅ Investigate **Data Anlysis** tab (see Notebook 01 – Regression Error Analysis for more information).
    
✅ Investigate **Feature importances** tab (see Notebook 01 – Regression Error Analysis for more information).
    
✅ Investigate **Counterfactuals** tab: </br>
&emsp; What-if allows you to perturb features for any input and observe how the model's prediction changes. To see how it works, first select a data point and then click on "Create what-if counterfactual". The first row of the table that will show up is original data point at selected index. The lines below of it show counterfactual examples. Mainly, they give you insight about minimum change in variable(s) to introduce to get opposite class prediction. </br>
&emsp; This is important in business perspective too. A householder may want to know what changes to do in his/her house to change classification label (from 'lower than median' to 'higher than median'). For short, Counterfactual tab provides us with actionable items to affect the outcome. For regression task, it works with ranges, i.e. minimal change than will move prediction from one range to another. 
    
</div>

See this [developer blog](https://techcommunity.microsoft.com/t5/ai-machine-learning-blog/responsible-ai-dashboard-a-one-stop-shop-for-operationalizing/ba-p/3030944) (Model Debugging Flow section) to learn more about this use case and how to use the dashboard to debug your housing price prediction model.

### References
- Responsible AI Toolbox [[Source]](https://github.com/microsoft/responsible-ai-toolbox/blob/main/notebooks/responsibleaidashboard/responsibleaidashboard-housing-classification-model-debugging.ipynb) 